In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

train_dir = '/kaggle/input/martianlunar-crater-detection-dataset/craters/train'
validation_dir = '/kaggle/input/martianlunar-crater-detection-dataset/craters/valid'



In [ ]:



train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)


In [ ]:

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

In [ ]:

history = model.fit(
    train_generator,
    steps_per_epoch=10,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=50
)

In [ ]:

loss, accuracy = model.evaluate(validation_generator)
print(f'Validation accuracy: {accuracy*100:.2f}%')


model.save('crater_detection_model.h5')

In [6]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    return img, edges

def detect_craters(edges):
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def analyze_craters(img, contours):
    crater_info = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        diameter = (w + h) / 2
        aspect_ratio = w / float(h)
        crater_info.append({
            'Bounding Box': (x, y, w, h),
            'Diameter': diameter,
            'Aspect Ratio': aspect_ratio
        })
        
        cv2.putText(img, f'D:{diameter:.1f}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.putText(img, f'AR:{aspect_ratio:.2f}', (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    return img, crater_info


In [7]:
def display_results(img, crater_info):
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Detected Craters')
    plt.axis('off')
    plt.show()
    
    for i, info in enumerate(crater_info):
        print(f"Crater {i + 1}:")
        print(f"  Bounding Box: {info['Bounding Box']}")
        print(f"  Diameter: {info['Diameter']} pixels")
        print(f"  Aspect Ratio: {info['Aspect Ratio']:.2f}")

In [ ]:

def process_dataset(directory):
    for root, _, files in os.walk(directory):
        for filename in files:
            if filename.endswith(".jpg") or filename.endswith(".png"):
                image_path = os.path.join(root, filename)
                img, edges = preprocess_image(image_path)
                contours = detect_craters(edges)
                img_with_craters, crater_info = analyze_craters(img, contours)
                display_results(img_with_craters, crater_info)


process_dataset(validation_dir)